# **Cat and Dog Image Project Report**


### **Abstract**

### **Introduction**

### **Background**

### **Data**

### **Methods**

BVOW - (Bag of Visual Words) is a method of image classification that builds a "vocabulary" of BRISK features to classify an image. When used for binary image classification, histograms of features are created to represent each image and a comparison between the classes using a KNN classifier is applied. The output is a predictor that will extract the feature "vocabulary" from the inferenced image and return a class prediction.


### **Evaluation**

### **Conclusion**